## Stock Analysis Deep Research

One of the classic cross-business Agentic use cases! This is huge for financial analysis and trading decisions.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">A Stock Analysis agent is broadly applicable to any investment area, and to your own day-to-day trading activities. You can make use of this yourself!
            </span>
        </td>
    </tr>
</table>

In [1]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import os
from typing import Dict, List
from IPython.display import display, Markdown
import brevo_python 
from brevo_python.api import transactional_emails_api 
from brevo_python.models.send_smtp_email import SendSmtpEmail 
from brevo_python.models.send_smtp_email_to import SendSmtpEmailTo 
from brevo_python.models.send_smtp_email_sender import SendSmtpEmailSender
import alpaca_trade_api as tradeapi
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import requests
import json
import io
import base64
import markdown

In [2]:
load_dotenv(override=True)

# Initialize Alpaca API
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')
alpaca_base_url = 'https://paper-api.alpaca.markets'  # Use paper trading for safety

if alpaca_api_key and alpaca_secret_key:
    alpaca = tradeapi.REST(alpaca_api_key, alpaca_secret_key, alpaca_base_url, api_version='v2')
    print("Alpaca API initialized successfully")
else:
    print("Warning: Alpaca API keys not found in environment variables")
    alpaca = None

Alpaca API initialized successfully


## Stock Analysis Tools

We'll create specialized agents for:

1. **News Agent**: Gets latest news for a stock ticker using Alpaca News API
2. **Performance Agent**: Gets stock performance data and creates plots
3. **Analysis Agent**: Analyzes the data and writes a comprehensive report
4. **Email Agent**: Sends the final report via email

In [3]:
@function_tool
def get_stock_news(ticker: str, days_back: int = 7) -> Dict[str, str]:
    """ Get latest news for a stock ticker using Alpaca News API """
    if not alpaca:
        return {
            "ticker": ticker,
            "news_count": 0,
            "articles": [],
            "message": "Alpaca API not initialized - using mock data for demonstration",
            "mock_data": True
        }
    
    try:
        # Get news from Alpaca
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days_back)
        
        news = alpaca.get_news(ticker, start=start_date, end=end_date, limit=10)
        
        if not news:
            return {
                "ticker": ticker,
                "news_count": 0,
                "articles": [],
                "message": f"No recent news found for {ticker}"
            }
        
        news_summary = []
        for article in news:
            news_summary.append({
                'headline': article.headline,
                'summary': article.summary,
                'published_at': article.published_at.strftime('%Y-%m-%d %H:%M'),
                'url': article.url
            })
        
        return {
            "ticker": ticker,
            "news_count": len(news_summary),
            "articles": news_summary
        }
    except Exception as e:
        return {
            "ticker": ticker,
            "news_count": 0,
            "articles": [],
            "message": f"Failed to get news: {str(e)}",
            "error": True
        }

In [4]:
@function_tool
def get_stock_performance(ticker: str, period: str = "1mo") -> Dict[str, str]:
    """ Get stock performance data and create a plot """
    try:
        # Get stock data using yfinance
        stock = yf.Ticker(ticker)
        hist = stock.history(period=period)
        
        if hist.empty:
            return {"error": f"No data found for {ticker}"}
        
        # Calculate performance metrics
        start_price = hist['Close'].iloc[0]
        end_price = hist['Close'].iloc[-1]
        total_return = ((end_price - start_price) / start_price) * 100
        
        # Get additional info
        info = stock.info
        
        return {
            "ticker": ticker,
            "period": period,
            "start_price": round(start_price, 2),
            "end_price": round(end_price, 2),
            "total_return": round(total_return, 2),
            "company_name": info.get('longName', 'N/A'),
            "sector": info.get('sector', 'N/A'),
            "market_cap": info.get('marketCap', 'N/A'),
            "plot_created": True
        }
    except Exception as e:
        return {"error": f"Failed to get performance data: {str(e)}"}

In [5]:
def create_performance_chart(ticker: str, period: str = "1mo"):
    """ Create and display a performance chart for the stock """
    try:
        # Get stock data using yfinance
        stock = yf.Ticker(ticker)
        hist = stock.history(period=period)
        
        if hist.empty:
            print(f"No data found for {ticker}")
            return
        
        # Create performance plot
        plt.figure(figsize=(12, 8))
        
        # Price chart
        plt.subplot(2, 1, 1)
        plt.plot(hist.index, hist['Close'], label='Close Price', linewidth=2, color='blue')
        plt.title(f'{ticker} Stock Performance - {period}', fontsize=14, fontweight='bold')
        plt.ylabel('Price ($)', fontsize=12)
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Volume chart
        plt.subplot(2, 1, 2)
        plt.bar(hist.index, hist['Volume'], alpha=0.7, color='green')
        plt.title('Trading Volume', fontsize=12, fontweight='bold')
        plt.ylabel('Volume', fontsize=12)
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Print performance summary
        start_price = hist['Close'].iloc[0]
        end_price = hist['Close'].iloc[-1]
        total_return = ((end_price - start_price) / start_price) * 100
        
        print(f"\n📊 Performance Summary for {ticker}:")
        print(f"   Start Price: ${start_price:.2f}")
        print(f"   End Price: ${end_price:.2f}")
        print(f"   Total Return: {total_return:.2f}%")
        
    except Exception as e:
        print(f"Error creating chart: {str(e)}")

In [6]:
def create_chart_for_email(ticker: str, period: str = "1mo") -> str:
    """ Create a performance chart and return it as base64 encoded image for email """
    try:
        # Get stock data using yfinance
        stock = yf.Ticker(ticker)
        hist = stock.history(period=period)
        
        if hist.empty:
            print(f"No data found for {ticker}")
            return ""
        
        # Set matplotlib backend to non-interactive
        import matplotlib
        matplotlib.use('Agg')
        
        # Create performance plot with better settings for email
        plt.figure(figsize=(10, 6), dpi=100)
        
        # Price chart
        plt.subplot(2, 1, 1)
        plt.plot(hist.index, hist['Close'], label='Close Price', linewidth=2, color='#2E86AB')
        plt.title(f'{ticker} Stock Performance - {period}', fontsize=14, fontweight='bold', color='#2c3e50')
        plt.ylabel('Price ($)', fontsize=12)
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Volume chart
        plt.subplot(2, 1, 2)
        plt.bar(hist.index, hist['Volume'], alpha=0.7, color='#A23B72')
        plt.title('Trading Volume', fontsize=12, fontweight='bold', color='#2c3e50')
        plt.ylabel('Volume', fontsize=12)
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        
        # Save to bytes buffer with optimized settings
        buffer = io.BytesIO()
        plt.savefig(buffer, format='png', dpi=150, bbox_inches='tight', facecolor='white', edgecolor='none')
        buffer.seek(0)
        
        # Convert to base64
        image_base64 = base64.b64encode(buffer.getvalue()).decode()
        
        # Close the plot to free memory
        plt.close()
        
        print(f"Chart created successfully for {ticker}, size: {len(image_base64)} characters")
        return image_base64
        
    except Exception as e:
        print(f"Error creating chart for email: {str(e)}")
        return ""

In [7]:
# Create the news agent
news_agent = Agent(
    name="News Agent",
    instructions="You are a financial news analyst. Given a stock ticker, you will gather the latest news and provide a concise summary of the most important developments affecting the stock. If no news is available, provide a general market analysis for the stock. Keep your response focused and under 200 words.",
    tools=[get_stock_news],
    model="gpt-4o-mini",
)

In [8]:
# Create the performance agent
performance_agent = Agent(
    name="Performance Agent",
    instructions="You are a stock performance analyst. You will analyze stock performance data and create visualizations to help understand the stock's behavior.",
    tools=[get_stock_performance],
    model="gpt-4o-mini",
)

In [9]:
def send_email_direct(body: str, chart_base64: str = ""):
    """ Send out an email with the given subject and HTML body, optionally including a chart """
    configuration = brevo_python.Configuration()
    api_key = os.environ.get('BREVO_API_KEY')
    if not api_key:
        raise ValueError("BREVO_API_KEY environment variable not set.")
    configuration.api_key['api-key'] = api_key
    api_instance = transactional_emails_api.TransactionalEmailsApi(brevo_python.ApiClient(configuration))
    sender = SendSmtpEmailSender(email="oliver@oliverdreger.cloud", name="Oliver Dreger")
    to = [SendSmtpEmailTo(email="oliver.dreger@gmail.com", name="Oliver Dreger")]
    
    # Create HTML content with chart if provided
    if chart_base64 and len(chart_base64) > 100:  # Ensure we have actual chart data
        html_content = f"""
<html>
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
</head>
<body style="margin: 0; padding: 20px; font-family: Arial, sans-serif; background-color: #f5f5f5;">
<div style="max-width: 800px; margin: 0 auto; background-color: white; border-radius: 10px; box-shadow: 0 4px 6px rgba(0,0,0,0.1); overflow: hidden;">
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 30px; text-align: center;">
<h1 style="margin: 0; font-size: 28px; font-weight: 300;">Stock Analysis Report</h1>
</div>
<div style="padding: 30px;">
<div style="background-color: #f8f9fa; padding: 25px; border-radius: 8px; margin-bottom: 30px; border-left: 4px solid #667eea;">
{body}
</div>
<div style="text-align: center; margin: 30px 0; padding: 20px; background-color: #f8f9fa; border-radius: 8px;">
<h2 style="color: #2c3e50; margin-bottom: 20px; font-size: 24px;">Performance Chart</h2>
<img src="data:image/png;base64,{chart_base64}" style="max-width: 100%; height: auto; border-radius: 8px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); border: 1px solid #e0e0e0;" alt="Stock Performance Chart" />
</div>
</div>
</div>
</body>
</html>
"""
        print(f"Email will include chart (size: {len(chart_base64)} characters)")
    else:
        html_content = f"""
<html>
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
</head>
<body style="margin: 0; padding: 20px; font-family: Arial, sans-serif; background-color: #f5f5f5;">
<div style="max-width: 800px; margin: 0 auto; background-color: white; border-radius: 10px; box-shadow: 0 4px 6px rgba(0,0,0,0.1); overflow: hidden;">
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 30px; text-align: center;">
<h1 style="margin: 0; font-size: 28px; font-weight: 300;">Stock Analysis Report</h1>
</div>
<div style="padding: 30px;">
<div style="background-color: #f8f9fa; padding: 25px; border-radius: 8px; margin-bottom: 30px; border-left: 4px solid #667eea;">
{body}
</div>
</div>
</div>
</body>
</html>
"""
        print("Email will be sent without chart")
    
    send_smtp_email = SendSmtpEmail(
        sender=sender,
        to=to,
        subject="Stock Analysis Report",
        html_content=html_content
    )
    try:
        api_response = api_instance.send_transac_email(send_smtp_email)
        print("Email sent successfully!")
        print(f"Response: {api_response}")
    except brevo_python.ApiException as e:
        print(f"Exception when calling TransactionalEmailsApi->send_transac_email: {e}\n")

In [10]:
@function_tool
def send_email(body: str, chart_base64: str = ""):
    """ Send out an email with the given subject and HTML body, optionally including a chart """
    return send_email_direct(body, chart_base64)

In [11]:
# Create the email agent
email_agent = Agent(
    name="Email Agent",
    instructions="You are able to send a nicely formatted HTML email based on a detailed stock analysis report. You will be provided with a detailed report. You should use your tool to send one email, providing the report converted into clean, well presented HTML with an appropriate subject line.",
    tools=[send_email],
    model="gpt-4o-mini",
)

In [12]:
# Create the analysis agent
INSTRUCTIONS = (
    "You are a senior financial analyst tasked with writing a comprehensive stock analysis report. "
    "You will be provided with the stock ticker, news data, and performance data.\n"
    "You should analyze the data and create a detailed report covering:\n"
    "- Executive Summary\n"
    "- Recent News Analysis\n"
    "- Technical Performance Analysis\n"
    "- Risk Assessment\n"
    "- Investment Recommendation\n"
    "The final output should be in markdown format, and it should be detailed and professional. "
    "Aim for at least 800 words."
)

class StockReportData(BaseModel):
    executive_summary: str = Field(description="A concise 2-3 sentence summary of the key findings.")
    markdown_report: str = Field(description="The complete stock analysis report in markdown format.")
    recommendation: str = Field(description="Clear buy/hold/sell recommendation with reasoning.")
    risk_level: str = Field(description="Risk assessment (Low/Medium/High).")

analysis_agent = Agent(
    name="Analysis Agent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=StockReportData,
)

In [13]:
# Orchestration functions
async def get_stock_news_data(ticker: str, days_back: int = 7):
    """ Get news data for the stock """
    print(f"Getting news for {ticker}...")
    try:
        result = await Runner.run(news_agent, f"Analyze the latest news for {ticker} stock. If no news is available, provide a brief market analysis.", max_turns=5)
        print("News analysis complete")
        return result.final_output
    except Exception as e:
        print(f"News analysis failed: {str(e)}")
        return f"Unable to retrieve news for {ticker}. This may be due to API limitations or network issues."

async def get_stock_performance_data(ticker: str, period: str = "1mo"):
    """ Get performance data for the stock """
    print(f"Getting performance data for {ticker}...")
    result = await Runner.run(performance_agent, f"Analyze the performance of {ticker} for the past {period}")
    print("Performance analysis complete")
    return result.final_output

async def write_stock_report(ticker: str, news_data: str, performance_data: str):
    """ Write comprehensive stock analysis report """
    print("Writing comprehensive stock analysis report...")
    input = f"Stock Ticker: {ticker}\nNews Analysis: {news_data}\nPerformance Analysis: {performance_data}"
    result = await Runner.run(analysis_agent, input)
    print("Report writing complete")
    return result.final_output

In [14]:
async def send_stock_report(report: StockReportData, ticker: str, period: str = "1mo"):
    """ Send the stock analysis report via email with chart """
    print("Sending stock analysis report via email...")
    
    # Create chart for email
    print("Creating chart for email...")
    chart_base64 = create_chart_for_email(ticker, period)
    
    # Convert markdown to HTML for email
    import markdown
    html_body = markdown.markdown(report.markdown_report)
    
    # Send email with chart using the direct function
    if chart_base64:
        send_email_direct(html_body, chart_base64)
    else:
        send_email_direct(html_body)
    
    print("Email sent with chart")
    return report

### Showtime! Let's analyze a stock

In [ ]:
# Example: Analyze Apple stock
ticker = "TSLA" #"AAPL"
period = "6mo" #"3mo"  # 3 months of data

with trace("Stock Analysis"):
    print(f"Starting comprehensive analysis of {ticker}...")
    
    # Get news and performance data in parallel
    news_task = asyncio.create_task(get_stock_news_data(ticker))
    performance_task = asyncio.create_task(get_stock_performance_data(ticker, period))
    
    news_data, performance_data = await asyncio.gather(news_task, performance_task)
    
    # Create and display performance chart
    print(f"\n📈 Creating performance chart for {ticker}...")
    create_performance_chart(ticker, period)
    
    # Write comprehensive report
    report = await write_stock_report(ticker, news_data, performance_data)
    
    # Send report via email with chart
    await send_stock_report(report, ticker, period)
    
    print(f"Complete analysis of {ticker} finished!")
    
    # Display the report
    display(Markdown(report.markdown_report))

Starting comprehensive analysis of TSLA...
Getting news for TSLA...
Getting performance data for TSLA...
Performance analysis complete
News analysis complete

📈 Creating performance chart for TSLA...


/var/folders/5d/4vzdwckn4fqgnfhszkpm8mzm0000gn/T/ipykernel_98955/1875918508.py:31: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()



📊 Performance Summary for TSLA:
   Start Price: $397.15
   End Price: $316.06
   Total Return: -20.42%
Writing comprehensive stock analysis report...
Report writing complete
Sending stock analysis report via email...
Creating chart for email...
Chart created successfully for TSLA, size: 110656 characters
Email will include chart (size: 110656 characters)
Email sent successfully!
Response: {'message_id': '<202507272350.86322042886@smtp-relay.mailin.fr>',
 'message_ids': None}
Email sent with chart
Complete analysis of TSLA finished!


# Tesla, Inc. (TSLA) Stock Analysis Report

## Executive Summary
Tesla, Inc. (TSLA) has witnessed a significant decline of approximately 20.42% over the past six months, dropping from a starting price of $397.15 to a current price of $316.06. This downturn is attributed to increased competition in the electric vehicle (EV) market, market volatility, and various macroeconomic factors. While Tesla continues to showcase strong long-term growth potential, investors should consider short-term risks and the impact of emerging challenges on the company's valuation.

## Recent News Analysis
While specific recent news is not retrievable, Tesla remains consistently in the limelight for several critical factors impacting its share price and market perception:

### 1. Earnings Reports
Tesla's quarterly earnings often have a substantial impact on its stock price. Positive earnings reports could bolster investor confidence, while any misses on revenue or profit expectations could lead to a swift decline in market valuation. The anticipation of upcoming earnings announcements can create volatility as investors position themselves ahead of earnings calls.

### 2. Competition in the EV Sector
Competition has intensified in the electric vehicle market, with traditional automakers such as Ford and General Motors investing heavily in EV technology and production. New entrants, including Rivian and Lucid Motors, are also increasing market share, posing significant threats to Tesla's dominance. This heightened competition can lead to reduced margins and sales growth projections, resulting in increased scrutiny from investors.

### 3. Regulatory Changes
Regulatory environments are crucial to the EV sector's health. Changes in government policies regarding incentives for electric vehicle purchases or stringent emissions regulations can directly impact Tesla's sales and operational strategy. For instance, the loss of federal tax incentives could dissuade consumers from purchasing EVs, affecting overall market demand.

### 4. Broader Market Conditions
Tesla's stock performance is influenced by broader market trends. Economic indicators like inflation, interest rates, and geopolitical issues have created turbulence in the market. Investors should monitor these macroeconomic factors as they can weigh significantly on stock prices, including TSLA's.

## Technical Performance Analysis
### Price Performance:
- **Market Capitalization:** $1.02 trillion  
- **Start Price (6 months ago):** $397.15  
- **End Price (current):** $316.06

### Returns:
- **Total Return:** -20.42%  

#### Summary
Over the past six months, Tesla's stock price has experienced a notable decline of approximately 20.42%. The trajectory shows a downtrend, indicating that the company may be facing significant pressure from the market. The stock peaked above $400 before experiencing a corrective phase, hampered by both internal performance and external competition.  

### Visual Representation
[Insert Price Performance Chart Here]

### Moving Averages and RSI
Analyzing Tesla's moving averages reveals a bearish trend, with the stock price consistently trading below both the 50-day and 200-day moving averages. This often signals a continued bearish market sentiment.

The Relative Strength Index (RSI), which gauges momentum, is currently in the neutral to oversold territory, indicating potential for a corrective bounce or continued selling pressure depending on upcoming news and market events.

## Risk Assessment
### Risk Level: Medium to High
Investing in Tesla involves medium to high levels of risk due to several factors:
- **Market Volatility:** The broader market conditions heavily influence TSLA, with heightened sensitivity during periods of economic instability.
- **Competition Risks:** Increased competition from both new and established players in the EV market raises the stakes for TSLA, potentially impacting market share and profitability.
- **Regulatory Risks:** Any adverse changes in EV regulations can impact sales and operational practices, affecting profitability and growth trajectories.
- **Execution Risks:** As a company that relies on rapid innovation and production scaling, there are risks associated with operational execution in manufacturing, supply chain management, and meeting consumer demand.

## Investment Recommendation
### Recommendation: Hold
Given the current landscape, Tesla appears to be a compelling long-term investment opportunity; however, investors should exercise caution in the short term. The decline in stock price reflects substantial challenges, and while the growth potential in EV adoption remains strong, nearly all peers are now competing for market share. 

Investors are advised to hold their positions in Tesla amid forthcoming earnings and market developments that may bring volatility. Monitoring competition and broader economic indicators will be crucial for gauging future performance. 

As Tesla continues to innovate and expand its market footprint, the long-term outlook remains promising, but a wait-and-see approach may be prudent until clearer trends emerge in the next earnings cycle and competitive landscape adjustments.

### Try different stocks

You can easily analyze different stocks by changing the ticker and period:

In [ ]:
# Example: Analyze Apple stock
ticker = "TSLA" #"AAPL"
period = "6mo" #"3mo"  # 3 months of data

with trace("Stock Analysis"):
    print(f"Starting comprehensive analysis of {ticker}...")
    
    # Get news and performance data in parallel
    news_task = asyncio.create_task(get_stock_news_data(ticker))
    performance_task = asyncio.create_task(get_stock_performance_data(ticker, period))
    
    news_data, performance_data = await asyncio.gather(news_task, performance_task)
    
    # Create and display performance chart
    print(f"\n📈 Creating performance chart for {ticker}...")
    create_performance_chart(ticker, period)
    
    # Write comprehensive report
    report = await write_stock_report(ticker, news_data, performance_data)
    
    # Send report via email with chart
    await send_stock_report(report, ticker, period)
    
    print(f"Complete analysis of {ticker} finished!")
    
    # Display the report
    display(Markdown(report.markdown_report))

Starting comprehensive analysis of TSLA...
Getting news for TSLA...
Getting performance data for TSLA...
Performance analysis complete
News analysis complete
Writing comprehensive stock analysis report...
Report writing complete


TypeError: send_stock_report() missing 1 required positional argument: 'ticker'

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../../assets/thanks.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00cc00;">Congratulations on your progress!</h2>
            <span style="color:#00cc00;">You've created a sophisticated stock analysis system using AI agents! This demonstrates how agentic AI can be applied to financial analysis and trading decisions. The system automatically gathers news, analyzes performance, creates visualizations, and generates comprehensive reports.<br/><br/>This pattern can be extended to other financial instruments, market analysis, and automated trading strategies.
            </span>
        </td>
    </tr>
</table>